In [ ]:
from tqdm import tqdm
from collections import Counter
from itertools import combinations
from nltk.stem import PorterStemmer
import joblib
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split

import imp
import copy
import pickle
import multiprocessing

import numpy as np
import pandas as pd
import utils as my_utils
import ELJST_script_BTM as lda
import matplotlib.pyplot as plt

In [ ]:
min_df = 5
max_df = .5
maxIters = 5

beta = .01
gamma = 10
n_topics = 5
lambda_param = 1.0

In [ ]:
dataset_name = "amazon_home_20000"

In [ ]:
dataset = pd.read_pickle("datasets/"+ dataset_name + "_" + "dataset")

In [ ]:
n_sentiment = dataset.sentiment.unique().shape[0]

In [ ]:
embedding_name = "fasttext_0.3"

In [ ]:
similar_words = pickle.load(open("resources/"+ dataset_name + "_" +embedding_name + ".pickle","rb"))

In [ ]:
%%time
for s in similar_words:
    for i in s.keys():
        k = list(set(s[i]))
        if i in k:
            k.remove(i)
        s[i] = k

In [ ]:
alpha = 0.1/n_topics * np.ones(n_topics)
gamma = [gamma/(n_topics*n_sentiment)]*n_sentiment

In [ ]:
imp.reload(lda)

In [ ]:
sampler = lda.SentimentLDAGibbsSampler(n_topics, alpha, beta, gamma, numSentiments=n_sentiment, 
                                       SentimentRange = n_sentiment, max_df = max_df, min_df = min_df, 
                                       lambda_param = lambda_param)

In [ ]:
%%time
sampler._initialize_(reviews = dataset.text.tolist(), labels = dataset.sentiment.tolist(), skipgramwindow=5)

In [ ]:
sampler.run(name=embedding_name, reviews=dataset.text.tolist(), labels=dataset.sentiment.tolist(), 
            similar_words=similar_words, mrf=True, maxIters=maxIters, debug=True)

In [ ]:
sampler.loglikelihood()

# Evaluations

In [ ]:
plt.plot(sampler.loglikelihood_history)

In [ ]:
silhouette_score(euclidean_distances(sampler.wordOccuranceMatrix),
                 sampler.dt_distribution.argmax(axis=1), metric='precomputed')

In [ ]:
davies_bouldin_score(sampler.wordOccuranceMatrix, sampler.dt_distribution.argmax(axis=1))

In [ ]:
my_utils.coherence_score(sampler.wordOccuranceMatrix, list(sampler.getTopKWords(5).values()), sampler.vocabulary)

In [ ]:
%%time
my_utils.get_hscore_multi(sampler.dt_distribution, sampler.wordOccuranceMatrix, n_topics, 500)

In [ ]:
sampler.loglikelihood()

In [ ]:
sampler.perplexity()

## Appendix

In [ ]:
# joblib.dump(sampler, "dumps/BTM_tester_" + embedding_name + "_" + str(lambda_param))

In [ ]:
# s = joblib.load("dumps/BTM_tester_" + embedding_name + "_" + str(lambda_param))